In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.__version__

Instructions for updating:
non-resource variables are not supported in the long term


'2.4.0'

In [2]:
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [3]:
# define reader
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.],[0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLineDataset`.


In [4]:
# collect batches of csv
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).


In [5]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [6]:
# Hypothesis
hypothesis = tf.matmul(X, W) + b

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize. Need a very small learning rate for this data set
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [7]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

In [8]:
# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [9]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 100 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  21176.459 
Prediction:
 [[ 2.1562152]
 [19.651169 ]
 [10.649282 ]
 [12.414448 ]
 [18.233244 ]
 [16.058672 ]
 [ 5.2259107]
 [-2.1672282]
 [22.987425 ]
 [24.18697  ]]
100 Cost:  70.068275 
Prediction:
 [[146.37837 ]
 [192.54028 ]
 [181.23323 ]
 [198.19014 ]
 [149.97948 ]
 [113.132126]
 [144.40796 ]
 [101.19841 ]
 [184.45583 ]
 [173.6986  ]]
200 Cost:  66.353966 
Prediction:
 [[146.59418]
 [192.33467]
 [181.27087]
 [198.22357]
 [149.71336]
 [112.87002]
 [144.55234]
 [101.48604]
 [184.1466 ]
 [173.354  ]]
300 Cost:  62.85966 
Prediction:
 [[146.80264]
 [192.13524]
 [181.30678]
 [198.25632]
 [149.45464]
 [112.61698]
 [144.69382]
 [101.76773]
 [183.84816]
 [173.0234 ]]
400 Cost:  59.571312 
Prediction:
 [[147.00406]
 [191.94176]
 [181.34102]
 [198.28842]
 [149.20311]
 [112.3727 ]
 [144.83243]
 [102.04359]
 [183.55998]
 [172.70613]]
500 Cost:  56.47632 
Prediction:
 [[147.19868 ]
 [191.75404 ]
 [181.37367 ]
 [198.31984 ]
 [148.95856 ]
 [112.13684 ]
 [144.96823 ]
 [102.313675]
 [183.2

In [10]:
coord.request_stop()
coord.join(threads)

In [11]:
# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Your score will be  [[205.65974]]
Other scores will be  [[164.3172 ]
 [172.66882]]


In [12]:
sess.close()